In [7]:
pip install  category_encoders




SyntaxError: invalid syntax (<ipython-input-7-3173b2c101c9>, line 1)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import  category_encoders as catec

ModuleNotFoundError: No module named 'category_encoders'

1- 'sex': sexo

2- 'age': idade

3- 'famsize': tamanho da família; LE3 - famílias com no máximo 3 pessoas, GT3 caso contrário

4- 'Pstatus': status de habitação com os pais; T - mora junto com os pais, A - mora separado;

5- 'traveltime': tempo de viagem entre escola e casa; 
    1 - <15 min., 2 - 15 a 30 min., 3 - 30 min. a 1 hora, or 4 - >1 hora

6- 'studytime': tempo de estudo semanal; 1 - <2 horas, 2 - 2 a 5 horas, 3 - 5 a 10 horas, ou 4 - >10 horas

7- 'schoolsup': suporte educacional extra escola

8- 'famsup': suporte educacional da família

9- 'paid': pagou aulas particulares por fora 

10- 'activities': atividades extra-curriculares

11- 'internet': acesso a internet em casa

12- 'romantic': se está num relacionamento

13- 'famrel': qualidade das relações familiares; 1 - muito ruim a 5 - muito boa

14- 'freetime': tempo livre após escola; 1 - muito pouco até 5 - muito

15- 'goout': relaciona-se com amigos; 1 - muito pouco até 5 - muito

16- 'absences': quantidade de faltas na escola

17- 'G3': nota média final no ensino médio


In [2]:
df=pd.read_csv("dataframe_exercicio_modulo_9.csv")


dict_={"no":0,"yes":1}
lista=["schoolsup","famsup","paid","activities","internet","romantic"]
for i in lista:
    df[i]=df[i].map(dict_)
df.sex=df.sex.map({"F":0,"M":1})    

In [3]:
df.head(5)

,sex,age,famsize,Pstatus,traveltime,studytime,schoolsup,famsup,paid,activities,internet,romantic,famrel,freetime,goout,absences,G3
0,0,18,GT3,A,2,2,1,0,0,0,0,0,4,3,4,6,6
1,0,17,GT3,T,1,2,0,1,0,0,1,0,5,3,3,4,6
2,0,15,LE3,T,1,2,1,0,1,0,1,0,4,3,2,10,10
3,0,15,GT3,T,1,3,0,1,1,1,1,1,3,2,2,2,15
4,0,16,GT3,T,1,2,0,1,1,0,0,0,4,3,2,4,10


In [4]:
df.isnull().sum()

sex           0
age           0
famsize       0
Pstatus       0
traveltime    0
studytime     0
schoolsup     0
famsup        0
paid          0
activities    0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
absences      0
G3            0
dtype: int64

#### Não foi constado dados faltantes.

In [5]:
df.corr()[["G3"]]

,G3
sex,-0.031472
age,-0.125282
traveltime,-0.102627
studytime,0.161629
schoolsup,-0.079346
famsup,0.013313
paid,-0.049388
activities,0.033998
internet,0.107064
romantic,-0.098363


In [6]:
ordi=catec.ordinal.OrdinalEncoder(cols=["famsize","Pstatus"])

NameError: name 'catec' is not defined

In [ ]:
ordi.fit(df)

In [ ]:
df_ord=ordi.transform(df)
df_ord.head()

In [ ]:
x=df.studytime.value_counts()
print(x)
type(x)

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Estudantes por tempo de estudo")
plt.bar(x.index,x,width=0.4)

plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,r2_score


In [ ]:
df_treino,df_teste=train_test_split(df_ord,test_size=0.25,random_state=0)

In [ ]:
df_treino.shape,df_teste.shape

In [ ]:
df.columns

In [ ]:
x_treino=df_treino.drop("G3",axis=1)
x_teste=df_teste.drop("G3",axis=1)

y_treino=df_treino.G3
y_teste=df_teste.G3
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def val_cruzada_r2(modelo,nome_modelo,x,y,num_folds):
    cvs=cross_val_score(estimator=modelo,
                       X=x,
                       y=y,
                       cv=num_folds,
                       scoring="r2")
    modelo.fit(x,y)
    r2_treino=r2_score(y,modelo.predict(x))
    plt.figure(figsize=[10,5])
    plt.scatter(range(1,1+num_folds),cvs,c="red",s=100)
    plt.plot(range(1,1+num_folds),cvs,c="red",lw=2,label="R2 Cruzado")
    plt.plot(range(1,1+num_folds),[r2_treino]*num_folds,c="blue",lw=2,label="R2 Treino")
    plt.xticks(range(1,1+num_folds))
    plt.yticks(np.arange(0,1.1,0.1))
    plt.legend(fontsize=8)
    plt.title(nome_modelo + " | R2 TREINO/VAL: "+ str(np.round(r2_treino,2)) +" / "+ str(np.round(cvs.mean(),2)),
              fontsize = 14, fontweight = 'bold')
    plt.show()

In [ ]:
val_cruzada_r2(LinearRegression(),"Regressão Linear",x_treino,y_treino,5)

In [ ]:
val_cruzada_r2(RandomForestRegressor(n_jobs=6, n_estimators=59, random_state=22),
               "Raandom Foreest",x_treino,y_treino,5)


In [ ]:
val_cruzada_r2(KNeighborsRegressor(n_neighbors=10),
               "Knn-5",
               x_treino,
               y_treino,5)

In [ ]:
val_cruzada_r2(DecisionTreeRegressor(max_depth=10),
              "max depth  1",
              x_treino,
              y_treino,
              5)

### após a análise dos algoritmos, vi que a regressão linear é a única que não está sofrendo overfittint,portanto será o modelo escolhido pra validação dos nossos dados.

In [ ]:
def rmse(ytrue,ypred):
    return np.sqrt(mean_squared_error(y_true=ytrue,y_pred=ypred))
    

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()

In [ ]:
lr.fit(x_treino,y_treino )

In [ ]:
lr.predict(x_treino)
print(r2_score(y_treino,lr.predict(x_treino)))

In [ ]:
y_pred1=lr.predict(x_teste)

In [ ]:
r2_teste=r2_score(y_teste,y_pred1)

In [ ]:
print(r2_teste)

### Dentro dos valores observados, vimos que a regressão linear previu bem os dados, não sofrendo overfitting.